# Exploratory Data Analysis - Steam Recommender System

**Course:** Recommendation Engines  
**Project:** Steam Video Games Recommendation System  
**Dataset:** Enhanced Steam dataset with derived features

---

## Objectives

This EDA focuses on understanding:
1. Dataset structure and quality
2. User behavior patterns and distributions
3. Game popularity and characteristics
4. Interaction patterns and implicit feedback
5. **Recommender-specific diagnostics:** sparsity, long-tail, cold-start potential
6. Data preparation for CF, CB, and hybrid models

---


## A. Load & Preview Dataset


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.sparse import csr_matrix
import warnings
warnings.filterwarnings('ignore')

# Set style for better-looking plots
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Set random seed for reproducibility
np.random.seed(42)

print("Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")


In [ ]:
# Load the enhanced dataset
df = pd.read_csv('steam-200k-enhanced-derived.csv')

print("="*80)
print("DATASET OVERVIEW")
print("="*80)
print(f"Dataset shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"\nColumn names ({len(df.columns)}):")
for i, col in enumerate(df.columns, 1):
    print(f"  {i:2d}. {col}")


In [ ]:
# Display first few rows
print("\nFirst 5 rows:")
display(df.head())

print("\nData types:")
print(df.dtypes)

print("\nMissing values:")
missing = df.isnull().sum()
if missing.sum() > 0:
    print(missing[missing > 0])
else:
    print("✓ No missing values found")


In [ ]:
# Basic statistics
print("="*80)
print("BASIC STATISTICS")
print("="*80)
print(f"\nUnique users: {df['user_id'].nunique():,}")
print(f"Unique games: {df['game_title'].nunique():,}")
print(f"Total interactions: {len(df):,}")

# Behavior breakdown
purchases = df[df['behavior_name'] == 'purchase']
plays = df[df['behavior_name'] == 'play']

print(f"\nBehavior breakdown:")
print(f"  - Purchases: {len(purchases):,} ({len(purchases)/len(df)*100:.1f}%)")
print(f"  - Plays: {len(plays):,} ({len(plays)/len(df)*100:.1f}%)")

# Unique user-game pairs
unique_pairs = df.groupby(['user_id', 'game_title']).size().shape[0]
print(f"\nUnique user-game pairs: {unique_pairs:,}")
print(f"Possible pairs (users × games): {df['user_id'].nunique() * df['game_title'].nunique():,}")
print(f"Sparsity (actual/potential): {unique_pairs / (df['user_id'].nunique() * df['game_title'].nunique()) * 100:.4f}%")


---

## B. EDA Sections

### 1. Basic Statistics


In [ ]:
# Summary statistics for numeric columns
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print("Numeric columns summary:")
display(df[numeric_cols].describe())


In [ ]:
# Categorical columns summary
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
print("Categorical columns:")
for col in categorical_cols:
    print(f"\n{col}:")
    print(f"  Unique values: {df[col].nunique()}")
    if df[col].nunique() <= 20:
        print(f"  Distribution:")
        print(df[col].value_counts().head(10))


### 2. User-Level Analysis


In [ ]:
# Create user-level aggregated dataframe
df_users = df.groupby('user_id').agg({
    'total_games_owned': 'first',
    'average_playtime': 'first',
    'total_playtime': 'first',
    'preferred_genre': 'first',
    'engagement_level': 'first'
}).reset_index()

print("="*80)
print("USER-LEVEL ANALYSIS")
print("="*80)
print(f"\nTotal unique users: {len(df_users):,}")
print(f"\nUser statistics:")
print(df_users.describe())


In [ ]:
# Games owned distribution
print("\nGames Owned Distribution:")
print(f"  Mean: {df_users['total_games_owned'].mean():.2f}")
print(f"  Median: {df_users['total_games_owned'].median():.2f}")
print(f"  Std: {df_users['total_games_owned'].std():.2f}")
print(f"  Min: {df_users['total_games_owned'].min()}")
print(f"  Max: {df_users['total_games_owned'].max()}")
print(f"  Q25: {df_users['total_games_owned'].quantile(0.25):.2f}")
print(f"  Q75: {df_users['total_games_owned'].quantile(0.75):.2f}")
print(f"  Q95: {df_users['total_games_owned'].quantile(0.95):.2f}")

# Users with few games (cold-start candidates)
users_few_games = (df_users['total_games_owned'] < 5).sum()
print(f"\n⚠️  Cold-start users (< 5 games): {users_few_games:,} ({users_few_games/len(df_users)*100:.1f}%)")
users_very_few = (df_users['total_games_owned'] < 3).sum()
print(f"⚠️  Very cold-start users (< 3 games): {users_very_few:,} ({users_very_few/len(df_users)*100:.1f}%)")


In [ ]:
# Total playtime distribution
print("\nTotal Playtime Distribution:")
print(f"  Mean: {df_users['total_playtime'].mean():.2f} hours")
print(f"  Median: {df_users['total_playtime'].median():.2f} hours")
print(f"  Max: {df_users['total_playtime'].max():.2f} hours")

# Average playtime per game
print(f"\nAverage Playtime per Game:")
print(f"  Mean: {df_users['average_playtime'].mean():.2f} hours")
print(f"  Median: {df_users['average_playtime'].median():.2f} hours")


In [ ]:
# Engagement level distribution
print("\nEngagement Level Distribution:")
engagement_dist = df_users['engagement_level'].value_counts()
for level, count in engagement_dist.items():
    print(f"  {level}: {count:,} ({count/len(df_users)*100:.1f}%)")

# Preferred genre distribution
print("\nPreferred Genre Distribution:")
genre_dist = df_users['preferred_genre'].value_counts()
for genre, count in genre_dist.head(10).items():
    print(f"  {genre}: {count:,} ({count/len(df_users)*100:.1f}%)")


In [ ]:
# Visualizations: User distributions
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. Games owned distribution
ax1 = axes[0, 0]
ax1.hist(df_users['total_games_owned'], bins=50, edgecolor='black', alpha=0.7)
ax1.set_xlabel('Total Games Owned')
ax1.set_ylabel('Number of Users')
ax1.set_title('Distribution of Games Owned per User')
ax1.set_yscale('log')
ax1.grid(True, alpha=0.3)

# 2. Games owned (log scale)
ax2 = axes[0, 1]
ax2.hist(np.log1p(df_users['total_games_owned']), bins=50, edgecolor='black', alpha=0.7, color='orange')
ax2.set_xlabel('Log(Games Owned + 1)')
ax2.set_ylabel('Number of Users')
ax2.set_title('Log-Transformed Games Owned Distribution')
ax2.grid(True, alpha=0.3)

# 3. Total playtime distribution
ax3 = axes[0, 2]
ax3.hist(df_users['total_playtime'], bins=50, edgecolor='black', alpha=0.7, color='green')
ax3.set_xlabel('Total Playtime (hours)')
ax3.set_ylabel('Number of Users')
ax3.set_title('Distribution of Total Playtime per User')
ax3.set_yscale('log')
ax3.grid(True, alpha=0.3)

# 4. Average playtime distribution
ax4 = axes[1, 0]
ax4.hist(df_users['average_playtime'], bins=50, edgecolor='black', alpha=0.7, color='purple')
ax4.set_xlabel('Average Playtime (hours)')
ax4.set_ylabel('Number of Users')
ax4.set_title('Distribution of Average Playtime per User')
ax4.set_yscale('log')
ax4.grid(True, alpha=0.3)

# 5. Engagement level distribution
ax5 = axes[1, 1]
engagement_counts = df_users['engagement_level'].value_counts()
engagement_counts.plot(kind='bar', ax=ax5, color='teal', edgecolor='black')
ax5.set_xlabel('Engagement Level')
ax5.set_ylabel('Number of Users')
ax5.set_title('User Distribution by Engagement Level')
ax5.tick_params(axis='x', rotation=45)
ax5.grid(True, alpha=0.3, axis='y')

# 6. Preferred genre distribution
ax6 = axes[1, 2]
genre_counts = df_users['preferred_genre'].value_counts().head(10)
genre_counts.plot(kind='bar', ax=ax6, color='coral', edgecolor='black')
ax6.set_xlabel('Preferred Genre')
ax6.set_ylabel('Number of Users')
ax6.set_title('Top 10 Preferred Genres')
ax6.tick_params(axis='x', rotation=45)
ax6.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('results/visualizations/user_distributions.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ Saved: results/visualizations/user_distributions.png")
